We are cleaning up the data

In [4]:
import pandas as pd
from pandas import DataFrame as df
import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import sqlite3

In [5]:
df = pd.read_csv("airport_weather_data.csv")

df.head()

,date,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,showers_sum,snowfall_sum,precipitation_hours,precipitation_probability_max,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,airport
0,2022-11-01 04:00:00+00:00,72.347900,52.187900,0.0,0.0,0.0,0.0,0.0,NaN,6.502709,12.750899,314.35632,MIA
1,2022-11-02 04:00:00+00:00,72.437900,56.867897,0.0,0.0,0.0,0.0,0.0,NaN,7.569527,12.750899,68.40039,MIA
2,2022-11-03 04:00:00+00:00,72.797900,53.987900,0.0,0.0,0.0,0.0,0.0,NaN,7.466353,13.198300,80.68073,MIA
3,2022-11-04 04:00:00+00:00,73.517900,53.357900,0.0,0.0,0.0,0.0,0.0,NaN,8.288983,15.882700,91.86882,MIA
4,2022-11-05 04:00:00+00:00,76.757904,60.647900,0.0,0.0,0.0,0.0,0.0,NaN,10.116089,24.383299,134.06648,MIA


Format the dat to MM-DD-YYYY, and bring the airport name to the front of the dataframe

In [6]:
df["date"] = pd.to_datetime(df["date"]) 
df["month"] = df["date"].dt.month       
df["day"] = df["date"].dt.day           
df["year"] = df["date"].dt.year          
df["day_of_week"] = df["date"].dt.dayofweek  

df.drop(columns=["date"], inplace=True)

df["precipitation_probability_max"].fillna(0, inplace=True) 
df = df[["airport"] + [col for col in df.columns if col != "airport"]]
df.head()

/var/folders/tq/0lh8dyqx1bn2g4vh2xspxhvm0000gn/T/ipykernel_1887/3057978549.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["precipitation_probability_max"].fillna(0, inplace=True)


,airport,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,showers_sum,snowfall_sum,precipitation_hours,precipitation_probability_max,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,month,day,year,day_of_week
0,MIA,72.347900,52.187900,0.0,0.0,0.0,0.0,0.0,0.0,6.502709,12.750899,314.35632,11,1,2022,1
1,MIA,72.437900,56.867897,0.0,0.0,0.0,0.0,0.0,0.0,7.569527,12.750899,68.40039,11,2,2022,2
2,MIA,72.797900,53.987900,0.0,0.0,0.0,0.0,0.0,0.0,7.466353,13.198300,80.68073,11,3,2022,3
3,MIA,73.517900,53.357900,0.0,0.0,0.0,0.0,0.0,0.0,8.288983,15.882700,91.86882,11,4,2022,4
4,MIA,76.757904,60.647900,0.0,0.0,0.0,0.0,0.0,0.0,10.116089,24.383299,134.06648,11,5,2022,5


Now, we need to normalize everything except the airport since we will need that to be able to join our database tables together

The different normalization for each variable, variables that have a 

In [7]:
# scaler_z = StandardScaler()
# df[["temperature_2m_max", "temperature_2m_min", "wind_speed_10m_max", "wind_gusts_10m_max"]] = scaler_z.fit_transform(
#     df[["temperature_2m_max", "temperature_2m_min", "wind_speed_10m_max", "wind_gusts_10m_max"]]
# )

# # Apply Min-Max Scaling
# scaler_mm = MinMaxScaler()
# df[["precipitation_sum", "rain_sum", "showers_sum", "snowfall_sum", "precipitation_hours", "precipitation_probability_max"]] = scaler_mm.fit_transform(
#     df[["precipitation_sum", "rain_sum", "showers_sum", "snowfall_sum", "precipitation_hours", "precipitation_probability_max"]]
# )

# # Convert wind direction to sine/cosine
# df["wind_direction_rad"] = np.radians(df["wind_direction_10m_dominant"])
# df["wind_direction_sin"] = np.sin(df["wind_direction_rad"])
# df["wind_direction_cos"] = np.cos(df["wind_direction_rad"])

# # Drop original wind direction column
# df.drop(columns=["wind_direction_10m_dominant", "wind_direction_rad"], inplace=True)

In [8]:
df.head()

,airport,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,showers_sum,snowfall_sum,precipitation_hours,precipitation_probability_max,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,month,day,year,day_of_week
0,MIA,72.347900,52.187900,0.0,0.0,0.0,0.0,0.0,0.0,6.502709,12.750899,314.35632,11,1,2022,1
1,MIA,72.437900,56.867897,0.0,0.0,0.0,0.0,0.0,0.0,7.569527,12.750899,68.40039,11,2,2022,2
2,MIA,72.797900,53.987900,0.0,0.0,0.0,0.0,0.0,0.0,7.466353,13.198300,80.68073,11,3,2022,3
3,MIA,73.517900,53.357900,0.0,0.0,0.0,0.0,0.0,0.0,8.288983,15.882700,91.86882,11,4,2022,4
4,MIA,76.757904,60.647900,0.0,0.0,0.0,0.0,0.0,0.0,10.116089,24.383299,134.06648,11,5,2022,5


In [9]:
# Create (or connect to) an SQLite database
db_name = "weather_data.db"  # Name of the database file
conn = sqlite3.connect(db_name)
cursor = conn.cursor()
# Save dataframe to SQLite table named 'weather'
df.to_sql("weather", conn, if_exists="replace", index=False)

# Close connection
conn.close()